<a href="https://colab.research.google.com/github/rlawogjs96/deeplearningnlp_starters/blob/main/GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. What is GloVe?

카운트 기반과 예측 기반을 모두 사용하는 단어 임베딩 방법론. Word2Vec 과 같이 뛰어난 성능을 보여준다.

In [3]:
pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 5.3 MB/s 


In [4]:
pip list | grep glove

glove-python-binary           0.2.0


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request 
import zipfile 
from lxml import etree 
import re 
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fccef1a2d50>)

In [9]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져옴.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text] 

In [10]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273380


In [11]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


glove.most_similar()는 입력 단어의 가장 유사한 단어들의 리스트를 리턴한다.

In [13]:
print(glove.most_similar("man"))
print(glove.most_similar("boy"))
print(glove.most_similar("water"))
print(glove.most_similar("physics"))
print(glove.most_similar("muscle"))
print(glove.most_similar("clean"))

[('woman', 0.9595697346515466), ('guy', 0.8837675555576077), ('girl', 0.8495041063180306), ('young', 0.8411860951269678)]
[('girl', 0.941511332305726), ('kid', 0.8285635486579125), ('woman', 0.825393266629894), ('man', 0.8217067921136911)]
[('fresh', 0.842281969254099), ('air', 0.8370272573269945), ('clean', 0.8216793282506704), ('electricity', 0.8144958679551221)]
[('chemistry', 0.8866871965443668), ('beauty', 0.878268287319308), ('economics', 0.8615575207744331), ('mathematics', 0.8507950597739462)]
[('tissue', 0.8367373598904942), ('nerve', 0.8214273364349003), ('skeletal', 0.7835144234788932), ('channel', 0.7523279257474466)]
[('water', 0.8216793282506706), ('fresh', 0.8010357861113125), ('wind', 0.7991583942969046), ('heat', 0.7961808789239007)]
